In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import glob
import os
from getpass import getpass

OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

In [8]:
%pip install pypdf
import os
from langchain.document_loaders import PyPDFLoader

documents = []
pdf_folder = "./resumes/"

for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file)
        
        loader = PyPDFLoader(pdf_path)

        document = loader.load()
        documents.append(document)



Note: you may need to restart the kernel to use updated packages.


In [10]:
llm = OpenAI()
def summarize_pdfs_from_folder(pdfs_folder):
    summaries = []
    for pdf_file in glob.glob(pdfs_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(docs)
        print("Summary for: ", pdf_file)
        print(summary)
        print("\n")
        summaries.append(summary)
    
    return summaries

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [6]:
def custom_summary(pdf_folder, custom_prompt):
    summaries = []
    for pdf_file in glob.glob(pdf_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        prompt_template = custom_prompt + """

        {text}

        SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
        chain = load_summarize_chain(llm, chain_type="map_reduce", 
                                    map_prompt=PROMPT, combine_prompt=PROMPT)
        summary_output = chain({"input_documents": docs},return_only_outputs=True)["output_text"]
        summaries.append(summary_output)
        
    return summaries

In [7]:
summaries = summarize_pdfs_from_folder("./resume")

In [8]:
import csv

data = []
for summary in summaries:
    lines = summary.split('\n')
    name = lines[0].split(": ")[1]
    summary_text = " ".join(lines[1:])
    data.append({"name": name, "summary": summary_text})

csv_filename = "resume_summaries.csv"

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['name', 'summary']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for entry in data:
        writer.writerow(entry)

print(f"CSV file '{csv_filename}' created successfully.")


CSV file 'resume_summaries.csv' created successfully.


In [9]:
# Save all summaries into one .txt file
with open("summaries.txt", "w") as f:
    for summary in summaries:
        f.write(summary + "\n"*3)

Storing Skills

In [10]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import PyPDFDirectoryLoader

In [11]:
loader = PyPDFDirectoryLoader("./resume/")

docs = loader.load()

# Create the vector store index
index = VectorstoreIndexCreator().from_loaders([loader])

ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [ ]:
import os
import re

cv_directory = "./resume/" 
cv_files = []

def remove_special_characters(text):
    return re.sub(r'[^\w\s]', '', text)

for filename in os.listdir(cv_directory):
    if filename.endswith(".pdf"):  
        pdf_name = os.path.splitext(filename)[0] 
        pdf_name = remove_special_characters(pdf_name)  
        pdf_name = pdf_name.replace("CV", "").replace("resume", "").replace("cv", "").replace("Resume", "").strip() 
        cv_files.append(pdf_name)

print("List of PDF file names:")
print(cv_files)

List of PDF file names:
['Pratishtha Singh', 'Sandali', 'Vani Seth']


In [ ]:
query = "List of skills of ('Vani Seth')?"

index.query(query)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-nSLZbSvqQViJHZZYqUphiWkz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-nSLZbSvqQViJHZZYqUphiWkz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

" Vani Seth's skills include Machine Learning, Artificial Intelligence, Data Science, Streamlit, Pandas, Numpy, Keras, Scikit-Learn, Matplotlib, Flask, MySql, Problem Solving, Time Management, Creative Thinking, Communication, Data Analytics, Statistics, and Content Writing."

In [ ]:
query = "What is the email of ('Sandali Gupta')?"

index.query(query)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-nSLZbSvqQViJHZZYqUphiWkz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-nSLZbSvqQViJHZZYqUphiWkz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

' sandaligupta1@email.com'